### 0、捷信数据

#### Overview

Many people struggle to get loans due to insufficient or non-existent credit histories. And, unfortunately, this population is often taken advantage of by untrustworthy lenders.

Home Credit strives to broaden financial inclusion for the unbanked population by providing a positive and safe borrowing experience. In order to make sure this underserved population has a positive loan experience, Home Credit makes use of a variety of alternative data--including telco and transactional information--to predict their clients' repayment abilities.

While Home Credit is currently using various statistical and machine learning methods to make these predictions, they're challenging Kagglers to help them unlock the full potential of their data. Doing so will ensure that clients capable of repayment are not rejected and that loans are given with a principal, maturity, and repayment calendar that will empower their clients to be successful.

#### Data Description

- application_{train|test}.csv
  - This is the main table, broken into two files for Train (with TARGET) and Test (without TARGET).
  - Static data for all applications. One row represents one loan in our data sample.
  
- bureau.csv
  - All client's previous credits provided by other financial institutions that were reported to Credit Bureau (for clients who have a loan in our sample).
  - For every loan in our sample, there are as many rows as number of credits the client had in Credit Bureau before the application date.

- bureau_balance.csv
  - Monthly balances of previous credits in Credit Bureau.
  - This table has one row for each month of history of every previous credit reported to Credit Bureau – i.e the table has (#loans in sample * # of relative previous credits * # of months where we have some history observable for the previous credits) rows.

- POS_CASH_balance.csv
  - Monthly balance snapshots of previous POS (point of sales) and cash loans that the applicant had with Home Credit.
  - This table has one row for each month of history of every previous credit in Home Credit (consumer credit and cash loans) related to loans in our sample – i.e. the table has (#loans in sample * # of relative previous credits * # of months in which we have some history observable for the previous credits) rows.

- credit_card_balance.csv
  - Monthly balance snapshots of previous credit cards that the applicant has with Home Credit.
  - This table has one row for each month of history of every previous credit in Home Credit (consumer credit and cash loans) related to loans in our sample – i.e. the table has (#loans in sample * # of relative previous credit cards * # of months where we have some history observable for the previous credit card) rows.

- previous_application.csv
  - All previous applications for Home Credit loans of clients who have loans in our sample.
  - There is one row for each previous application related to loans in our data sample.

- installments_payments.csv
  - Repayment history for the previously disbursed credits in Home Credit related to the loans in our sample.
  - There is a) one row for every payment that was made plus b) one row each for missed payment.
  - One row is equivalent to one payment of one installment OR one installment corresponding to one payment of one previous Home Credit credit related to loans in our sample.

- HomeCredit_columns_description.csv
  - This file contains descriptions for the columns in the various data files.

![home_credit](../img/home_credit.png)

In [19]:
import pandas as pd
import numpy as np
import featuretools as ft

pd.set_option('display.max_info_columns', 200)

In [4]:
#使用pandas读取7个数据表生成dataframe
data_path = '../data/捷信数据/'
app_train = pd.read_csv(data_path + 'application_train.csv')
app_test = pd.read_csv(data_path + 'application_test.csv')
bureau = pd.read_csv(data_path + 'bureau.csv')
bureau_balance = pd.read_csv(data_path + 'bureau_balance.csv')
cash = pd.read_csv(data_path + 'POS_CASH_balance.csv')
credit = pd.read_csv(data_path + 'credit_card_balance.csv')
previous = pd.read_csv(data_path + 'previous_application.csv')
installments =pd.read_csv(data_path + 'installments_payments.csv')

app_test['TARGET'] = np.nan
app = app_train.append(app_test, ignore_index = True, sort = True)

In [20]:
app.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 356255 entries, 0 to 356254
Data columns (total 122 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   AMT_ANNUITY                   356219 non-null  float64
 1   AMT_CREDIT                    356255 non-null  float64
 2   AMT_GOODS_PRICE               355977 non-null  float64
 3   AMT_INCOME_TOTAL              356255 non-null  float64
 4   AMT_REQ_CREDIT_BUREAU_DAY     308687 non-null  float64
 5   AMT_REQ_CREDIT_BUREAU_HOUR    308687 non-null  float64
 6   AMT_REQ_CREDIT_BUREAU_MON     308687 non-null  float64
 7   AMT_REQ_CREDIT_BUREAU_QRT     308687 non-null  float64
 8   AMT_REQ_CREDIT_BUREAU_WEEK    308687 non-null  float64
 9   AMT_REQ_CREDIT_BUREAU_YEAR    308687 non-null  float64
 10  APARTMENTS_AVG                176307 non-null  float64
 11  APARTMENTS_MEDI               176307 non-null  float64
 12  APARTMENTS_MODE               176307 non-nu

In [29]:
#有几张表的索引字段数据类型是浮点类型，需要把它们转换为整型，以确保可以正常地添加关系。(本身就是整型)
for index in ['SK_ID_CURR', 'SK_ID_PREV', 'SK_ID_BUREAU']:
    for dataset in [app, bureau, bureau_balance, cash, credit, previous, installments]:
        if index in list(dataset.columns):
            dataset[index] = dataset[index].fillna(0).astype(np.int32)

In [60]:
for dataset in [app, bureau, bureau_balance, cash, credit, previous, installments]:
    for index in list(dataset.columns):
        if dataset[index].dtype == 'float64':
            dataset[index] = dataset[index].fillna(0).astype(np.float32)
        elif dataset[index].dtype == 'int64':
            dataset[index] = dataset[index].fillna(0).astype(np.int32)

MemoryError: Unable to allocate array with shape (65, 356255) and data type float64

In [61]:
for dataset in [app, bureau, bureau_balance, cash, credit, previous, installments]:
    del dataset

### 2、构建实体和实体集

In [30]:
es = ft.EntitySet(id = 'clients')

import featuretools.variable_types as vtypes

app_types = {}

In [36]:
# 将两种类别的变量调整为布尔型
for col in app:
    if (app[col].nunique() == 2) and (app[col].dtype == float):
        app_types[col] = vtypes.Boolean
        
# 剔除目标变量 `TARGET`
del app_types['TARGET']

In [38]:
#某些类别变量的业务含义上还有排序性，需要配置为Ordinal类型。
app_types['REGION_RATING_CLIENT'] = vtypes.Ordinal
app_types['REGION_RATING_CLIENT_W_CITY'] = vtypes.Ordinal
app_types['HOUR_APPR_PROCESS_START'] = vtypes.Ordinal

In [40]:
installments.head()

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,1054186,161674,1.0,6,-1180.0,-1187.0,6948.360,6948.360
1,1330831,151639,0.0,34,-2156.0,-2156.0,1716.525,1716.525
2,2085231,193053,2.0,1,-63.0,-63.0,25425.000,25425.000
3,2452527,199697,1.0,3,-2418.0,-2426.0,24350.130,24350.130
4,2714724,167756,1.0,2,-1383.0,-1366.0,2165.040,2160.585


In [41]:
previous_types = {}
for col in previous:
    if (previous[col].nunique() == 2) and (previous[col].dtype == float):
        previous_types[col] = vtypes.Boolean
        
installments = installments.drop(columns = ['SK_ID_CURR'])
credit = credit.drop(columns = ['SK_ID_CURR'])
cash = cash.drop(columns = ['SK_ID_CURR'])

In [43]:
# 创建具有唯一标识索引的实体
es = es.entity_from_dataframe(entity_id='app',
                              dataframe=app,
                              index='SK_ID_CURR',
                              variable_types=app_types)

es = es.entity_from_dataframe(entity_id='bureau',
                              dataframe=bureau,
                              index='SK_ID_BUREAU')

es = es.entity_from_dataframe(entity_id='previous',
                              dataframe=previous,
                              index='SK_ID_PREV',
                              variable_types=previous_types)

# 创建没有唯一标识索引的实体
es = es.entity_from_dataframe(entity_id='bureau_balance',
                              dataframe=bureau_balance,
                              make_index=True,
                              index='bureaubalance_index')

es = es.entity_from_dataframe(entity_id='cash',
                              dataframe=cash,
                              make_index=True,
                              index='cash_index')

es = es.entity_from_dataframe(entity_id='installments',
                              dataframe=installments,
                              make_index=True,
                              index='installments_index')

es = es.entity_from_dataframe(entity_id='credit',
                              dataframe=credit,
                              make_index=True,
                              index='credit_index')

In [44]:
# 输出实体集信息
es

Entityset: clients
  Entities:
    app [Rows: 356255, Columns: 122]
    bureau [Rows: 1716428, Columns: 17]
    previous [Rows: 1670214, Columns: 37]
    bureau_balance [Rows: 27299925, Columns: 4]
    cash [Rows: 10001358, Columns: 8]
    installments [Rows: 13605401, Columns: 8]
    credit [Rows: 3840312, Columns: 23]
  Relationships:
    No relationships

### 3、构建关系，添加到EntitySet中

In [45]:
# 输出
print('Parent: app, Parent Variable of bureau: SK_ID_CURR\n\n',
      app.iloc[:, 111:114].head())  #111:115
print('\nChild: bureau, Child Variable of app: SK_ID_CURR\n\n',
      bureau[bureau['SK_ID_CURR'] == 100002].iloc[:, :3])

Parent: app, Parent Variable of bureau: SK_ID_CURR

    SK_ID_CURR  TARGET  TOTALAREA_MODE
0      100002     1.0          0.0149
1      100003     0.0          0.0714
2      100004     0.0             NaN
3      100006     0.0             NaN
4      100007     0.0             NaN

Child: bureau, Child Variable of app: SK_ID_CURR

          SK_ID_CURR  SK_ID_BUREAU CREDIT_ACTIVE
675684       100002       6158904        Closed
675685       100002       6158905        Closed
675686       100002       6158906        Closed
675687       100002       6158907        Closed
675688       100002       6158908        Closed
675689       100002       6158909        Active
1337779      100002       6158903        Active
1486113      100002       6113835        Closed


In [46]:
# 为app 和bureau构建关联关系
r_app_bureau = ft.Relationship(es['app']['SK_ID_CURR'],
                               es['bureau']['SK_ID_CURR'])

# 为bureau和bureau _balance构建关联关系
r_bureau_balance = ft.Relationship(es['bureau']['SK_ID_BUREAU'],
                                   es['bureau_balance']['SK_ID_BUREAU'])

# 为app 和previous构建关联关系
r_app_previous = ft.Relationship(es['app']['SK_ID_CURR'],
                                 es['previous']['SK_ID_CURR'])

# 为previous与 cash、 installments、credit 构建关联关系
r_previous_cash = ft.Relationship(es['previous']['SK_ID_PREV'],
                                  es['cash']['SK_ID_PREV'])
r_previous_installments = ft.Relationship(es['previous']['SK_ID_PREV'],
                                          es['installments']['SK_ID_PREV'])
r_previous_credit = ft.Relationship(es['previous']['SK_ID_PREV'],
                                    es['credit']['SK_ID_PREV'])

# 构建好的关系添加到实体集
es = es.add_relationships([r_app_bureau, r_bureau_balance, r_app_previous,
                           r_previous_cash, r_previous_installments, r_previous_credit])

In [47]:
es

Entityset: clients
  Entities:
    app [Rows: 356255, Columns: 122]
    bureau [Rows: 1716428, Columns: 17]
    previous [Rows: 1670214, Columns: 37]
    bureau_balance [Rows: 27299925, Columns: 4]
    cash [Rows: 10001358, Columns: 8]
    installments [Rows: 13605401, Columns: 8]
    credit [Rows: 3840312, Columns: 23]
  Relationships:
    bureau.SK_ID_CURR -> app.SK_ID_CURR
    bureau_balance.SK_ID_BUREAU -> bureau.SK_ID_BUREAU
    previous.SK_ID_CURR -> app.SK_ID_CURR
    cash.SK_ID_PREV -> previous.SK_ID_PREV
    installments.SK_ID_PREV -> previous.SK_ID_PREV
    credit.SK_ID_PREV -> previous.SK_ID_PREV

### 4、特征基元

In [62]:
# 自定义函数
def plusOne(column):
    return column + 1

# 通过接口添加自定义基元
plus_one = ft.primitives.make_trans_primitive(function=plusOne,
                                              input_types=[ft.variable_types.Numeric],
                                              return_type=ft.variable_types.Numeric)

feature_matrixp, feature_namesp = ft.dfs(entityset=es,
                                         target_entity='app',
                                         trans_primitives=[plus_one],
                                         agg_primitives=['count'],
                                         max_depth=2)

MemoryError: Unable to allocate array with shape (40, 356255) and data type int64

In [ ]:
# 查看两组比对衍生特征
feature_matrixp[['AMT_ANNUITY','PLUSONE(AMT_ANNUITY)','COUNT(bureau)','PLUSONE(COUNT(bureau))']].head()

### 5、深度特征合成

#### 使用dfs默认基元合成特征

In [49]:
feature_names = ft.dfs(entityset=es,
                       target_entity='app',
                       features_only=True)

In [ ]:
print(len(feature_names))

In [ ]:
feature_names[1000:1010]

In [ ]:
# 选择运算基元
agg_primitives = ["sum", "max", "min", "mean", "count", "percent_true", "num_unique", "mode"]
trans_primitives = ['percentile', 'and']

# 调用dfs接口 
feature_matrix, feature_names = ft.dfs(entityset=es,
                                      target_entity='app',
                                      agg_primitives=agg_primitives,
                                      trans_primitives=trans_primitives,
                                      features_only=False,
                                      max_depth=2)